## Ingest drivers.json file
Step 1- Read the json file using the spark dataframe reader API

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType, StructField, TimestampType, DateType
from pyspark.sql.functions import col, struct,current_timestamp , concat, lit

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType(), True), StructField("surname", StringType(), True)])

In [0]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema, True),
                                    #StructField("surname", StringType(), True),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)])

In [0]:
# Load the JSON file
drivers_df = spark.read\
    .schema(drivers_schema)\
    .json(f"{raw_folder_path}/{v_file_date}/drivers.json")

In [0]:
# Transform the DataFrame: Combine forename and surname into a nested "name" struct
#drivers_df = drivers_df.withColumn("name", struct(col("forename"), col("surname"))) \
                  # .drop("forename", "surname")  # Drop old unnested columns

In [0]:
display(drivers_df)

In [0]:
drivers_df= drivers_df.select(col("driverId"), col("driverRef"), col("number"), col("code"), col("name"), col("dob"), col("nationality"))

In [0]:
drivers_df.printSchema()

#### Step 2 - Rename columns and add ingestion date

In [0]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId","driver_id")\
    .withColumnRenamed("driverRef","driver_ref")\
    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))\
    .withColumn("data_source", lit(v_data_source))\
    .withColumn("file_date", lit(v_file_date)) 

In [0]:
drivers_with_columns_df =add_ingestion_date(drivers_with_columns_df)

In [0]:
display(drivers_with_columns_df)

#### Step 3 - Drop unwanted columns from the dataframe

In [0]:
drivers_final_df =drivers_with_columns_df.drop(col("url"))


#### Step 4 - Write to output to processed container in parquet format

In [0]:
##drivers_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/drivers")
#drivers_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.drivers")
drivers_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.drivers")

In [0]:
%fs
ls /mnt/formula1dl2025practice/processed/drivers

In [0]:
dbutils.notebook.exit("Success")